*LIBRERIAS, VARIABLES Y FUNCIONES*

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime 

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
final_table_attribs = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'
log_file = 'code_log.txt'
exchange_rate_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    tables = soup.find_all('table')
    target_table = tables[0]
    rows = target_table.find_all('tr')
    data_list = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) != 0:  # Omitir encabezados o filas vacías
            data_dict = {
                "Name": cols[1].text.strip(),  # Nombre del banco
                "MC_USD_Billion": cols[2].text.strip()  # Market cap en USD
            }
            data_list.append(data_dict)
    df = pd.DataFrame(data_list, columns=table_attribs)
    return df

def transform(df, exchange_rate_url):
    exchange_rates_df = pd.read_csv(exchange_rate_url)
    exchange_rate = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate']))
    df['MC_USD_Billion'] = pd.to_numeric(df['MC_USD_Billion'], errors='coerce')
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion'].fillna(0)]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion'].fillna(0)]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion'].fillna(0)]
    log_progress("Transformación de datos completada - Nuevas columnas agregadas")  
    return df

def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)
    log_progress("Datos cargados correctamente en el archivo CSV.")

def load_to_db(conn, table_name, df):
    try:
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        log_progress("Datos cargados correctamente en la base de datos SQLite3.")
        print("Carga completada en la base de datos SQLite3.")
    except Exception as e:
        log_progress(f"Error al cargar en la base de datos: {str(e)}")
        print(f"Error al cargar en la base de datos: {str(e)}")

def run_queries(query, conn):
    try:
        result = conn.execute(query).fetchall()
        log_progress(f"Ejecutando consulta: {query}")
        print(f"\nConsulta: {query}")
        print("Resultado:")
        for row in result:
            print(row)
    except Exception as e:
        log_progress(f"Error al ejecutar la consulta: {str(e)}")
        print(f"Error: {str(e)}")

*-----------------------------------------------*

*-----------------------------------------------*

*PROBAR FUNCIONES DE A UNO*

In [35]:
# Probar log_progress
log_progress("Prueba de progreso - Inicio de prueba")
log_progress("Prueba de progreso - Fin de prueba")
print("Mensajes de progreso registrados en 'code_log.txt'")


Mensajes de progreso registrados en 'code_log.txt'


*-----------------------------------------------*

*-----------------------------------------------*

In [36]:
# Probar extract
test_df = extract('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks', ["Name", "MC_USD_Billion"])
print("Extracción completada. Aquí están las primeras filas de los datos:")
print(test_df.head())  # Mostrar las primeras filas para validar

Extracción completada. Aquí están las primeras filas de los datos:
                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91


*-----------------------------------------------*

*-----------------------------------------------*

In [37]:
# Llamar a la función transform
df_transformed = transform(test_df, exchange_rate_url)
print(df_transformed.head())

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75  
3          149.43        13328.41  
4          146.86        13098.63  


*-----------------------------------------------*

*-----------------------------------------------*

In [38]:
# Llamar a la función para cargar los datos transformados a un archivo CSV
load_to_csv(df_transformed, csv_path)
# Imprimir un mensaje para verificar
print(f"Datos guardados en {csv_path}")

Datos guardados en ./Largest_banks_data.csv


*-----------------------------------------------*

*-----------------------------------------------*

In [40]:
import sqlite3
conn = sqlite3.connect(db_name)  # Usa 'Banks.db' como se menciona en la tarea
load_to_db(conn, table_name, df_transformed)
conn.close()
print("Conexión cerrada con la base de datos SQLite3.")

Carga completada en la base de datos SQLite3.
Conexión cerrada con la base de datos SQLite3.


*-----------------------------------------------*

*-----------------------------------------------*

In [42]:
# Crear la conexión a la base de datos SQLite3
conn = sqlite3.connect(db_name)

# Consulta 1: Imprimir todo el contenido de la tabla Largest_banks
query_1 = "SELECT * FROM Largest_banks"
run_queries(query_1, conn)

# Consulta 2: Imprimir la capitalización de mercado promedio en mil millones de GBP
query_2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_queries(query_2, conn)

# Consulta 3: Imprimir solo los nombres de los 5 principales bancos
query_3 = "SELECT Name FROM Largest_banks LIMIT 5"
run_queries(query_3, conn)

# Cerrar la conexión a la base de datos después de ejecutar las consultas
conn.close()
print("\nConexión cerrada con la base de datos SQLite3.")



Consulta: SELECT * FROM Largest_banks
Resultado:
('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)

Consulta: SELECT AVG(MC_GBP_Billion) FROM Largest_banks
Resultado:
(151.987,)

Consulta: SELECT Name FROM Largest_banks LIMIT 5
Resultado:
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)

Conexión cerrada con la base de datos SQLite3.


*-----------------------------------------------*

*-----------------------------------------------*

*LOG*

In [44]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, exchange_rate_url)
log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated.')

load_to_db(sql_connection, table_name, df)
log_progress('Data loaded to Database as table. Running the query')

query_statement_1 = f"SELECT * FROM {table_name}"
query_statement_2 = f"SELECT AVG(MC_USD_Billion) FROM {table_name}"
query_statement_3 = f"SELECT Name FROM {table_name} LIMIT 5"

run_queries(query_statement_1, sql_connection)
run_queries(query_statement_2, sql_connection)
run_queries(query_statement_3, sql_connection)

log_progress('Process Complete.')

sql_connection.close()
log_progress('SQL Connection closed.')

Carga completada en la base de datos SQLite3.

Consulta: SELECT * FROM Largest_banks
Resultado:
('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)

Consulta: SELECT AVG(MC_USD_Billion) FROM Largest_banks
Resultado:
(189.98200000000003,)

Consulta: SELECT Name FROM Largest_banks LIMIT 5
Resultado:
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)
